# 1. Load LLM and define Cypher translation prompt

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Replace with your actual API key or environment setup
llm = ChatOpenAI(temperature=0, model="gpt-4", openai_api_key="sk-xxx")

schema_context = '''
Nodes:
- Doctor {npi, specialty, credential, grad_year, gender}
  • gender: 'M' or 'F'
  • grad_year: integer
- Facility {name, city, state, zip_code, adi_nat_2020}
- MedicalSchool {name}
- JobMobility {moves: integer}

Relationships:
(:Doctor)-[:WORKS_AT]->(:Facility)
(:Doctor)-[:GRADUATED_FROM]->(:MedicalSchool)
(:Doctor)-[:HAS_MOBILITY]->(:JobMobility)

Guidelines:
- Do not use SQL syntax like GROUP BY. Use WITH instead.
- Always assume 'F' or 'M' for gender values.
- High ADI means f.adi_nat_2020 >= 80
- Prefer COUNT() or AVG()
- Use EXISTS or IS NOT NULL when needed
'''

cypher_prompt = PromptTemplate(
    input_variables=["question"],
    template="""
You are a Cypher expert helping users query a healthcare knowledge graph.

Schema:
{schema}

User question: {question}

Return only the Cypher query.
""".strip()
).partial(schema=schema_context)

cypher_chain = LLMChain(llm=llm, prompt=cypher_prompt)

# Try a sample question
question = "Which states have the highest average ADI score?"
cypher_query = cypher_chain.run(question=question).strip()
print(cypher_query)